In [78]:
import pandas as pd
import itertools

In [79]:
#Read in CSV file with coding
df = pd.read_csv("Final Coding.csv",keep_default_na=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129 entries, 0 to 1128
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    1129 non-null   int64 
 1   tweet_id                      1129 non-null   int64 
 2   user_username                 1129 non-null   object
 3   text                          1129 non-null   object
 4   created_at                    1129 non-null   object
 5   conversation_id               1129 non-null   int64 
 6   lang                          1129 non-null   object
 7   possibly_sensitive            1129 non-null   bool  
 8   author_id                     1129 non-null   int64 
 9   source                        1129 non-null   object
 10  in_reply_to_user_id           1129 non-null   object
 11  user_verified                 1129 non-null   bool  
 12  user_profile_image_url        1129 non-null   object
 13  user_name         

In [80]:
df = pd.read_csv("Final Coding.csv",keep_default_na=False)
#The codes were in 5 different columns; lets aggregate them into a new column
df["all codes"] = df[["SO code","BC code","BtC code","CCM code","SJEDI code"]].agg(';'.join,axis=1)
#We separated them by ;, so convert to an array 
df["all codes"] = df["all codes"].apply(lambda s: s.split(';'))
#Strip out leading/trailing spaces
df["all codes"] = df["all codes"].apply(lambda s: [t.strip() for t in s if t])
#Sort the arrays, so we can be confident the same things are not listed in two different ways
df["all codes"] = df["all codes"].apply(sorted)
#Convert back to string with ; between the codes
df["all codes"] = df["all codes"].apply(lambda s: ';'.join(s))

df_names = pd.read_csv("Names.csv")
df = df.merge(df_names,left_on='user_username',right_on='User Name',how='left')
df=df.rename(columns={'In JMM 2020 Program?': 'At JMM 2020', 'In JMM 2021 Program': 'At JMM 2021', 'Too anonymous to determine':'Anonymous'})

#print(list(df.columns))
#df[['User Name', 'Display Name', 'JMM 2020', 'JMM 2021', 'Anonymous']]
users = df[['User Name', 'Display Name', 'author_id','At JMM 2020','At JMM 2021', 'Anonymous', 'Organization']].groupby(by=['User Name']).head(1)
users

#Check merge
print('There were ' + str(len(df['Display Name'].unique())) + ' users in the dataset')



#Get list of the codes
codes = df['all codes'].apply(lambda s: s.split(';')).explode().unique()
#Make a new column for each code, and mark it true/false if that code appears in 'all codes' column
for code in codes:
    df[code] = df['all codes'].apply(lambda s: code in s)


#Make dictionaries of user_ids to usernamess, and tweetids to usernames
users=df[['user_username','author_id']].drop_duplicates()
user_dict = dict(zip(users['author_id'],users['user_username']))

tweet_dict=dict(zip(df['tweet_id'],df['user_username']))


#Lookup functions for tweetids and userids
def tweeter_lookup(tweetid):
    if tweetid == "":
        return ""
    elif int(tweetid) in tweet_dict.keys():
        return tweet_dict[int(tweetid)]
    else:
        return ""
    
def lookup_username(s):
    if s=='':
        return ''
    elif int(s) not in user_dict.keys():
        return "Error with id "+str(s)
    else:
        return user_dict[int(s)]

#Get receivers for replies
df['Receiver']=''
df['Receiver'].mask(df['in_reply_to_user_id']!='NA',df['in_reply_to_user_id'],inplace=True)
df['Receiver'] = df['Receiver'].apply(lookup_username)

#Get recievers for quote tweets and retweets
df['sourcetweeter']=''
df['sourcetweeter'].mask(df['sourcetweet_type']=='quoted', df['sourcetweet_id'],inplace=True )
df['sourcetweeter'].mask(df['sourcetweet_type']=='retweeted', df['sourcetweet_id'],inplace=True )
df['sourcetweeter']=df['sourcetweeter'].apply(tweeter_lookup)
df['Receiver'].mask(df['sourcetweeter']!="",df['sourcetweeter'],inplace=True)


df['date']=df['created_at'].apply(lambda s: str(s)[0:10])
df[df['date']>'2020-01-16'].head(20)
time_bins = ['Pre JMM 2020', 'JMM 2020', 'Inter JMM', 'JMM 2021', 'Post JMMs']
#JMM 2020 time inverval: Jan 12 -- Jan 25
#JMM 2021 time interval: Jan 3 -- Jan 16
df[time_bins[0]]=False
df[time_bins[0]].mask(df['date']<'2020-01-12',True,inplace=True)
df[time_bins[1]]=False
df[time_bins[1]].mask((df['date']>='2020-01-12') & (df['date']<='2020-01-25'),True,inplace=True)
df[time_bins[2]]=False
df[time_bins[2]].mask((df['date']>'2020-01-25') & (df['date']<'2021-01-03'),True,inplace=True)
df[time_bins[3]]=False
df[time_bins[3]].mask((df['date']>='2021-01-03') & (df['date']<='2021-01-16'),True,inplace=True)
df[time_bins[4]]=False
df[time_bins[4]].mask(df['date']>'2021-01-16',True,inplace=True)

#Clean up the frame a bit
df=df.drop(columns=['sourcetweeter'])



df.to_csv("all-data.csv")

#users

There were 327 users in the dataset


## Codes by time bucket

In [81]:
list=[]
for time in time_bins:
   list.append(df.groupby([time]).sum()[codes].transpose()[True])

newdf=pd.DataFrame(list)
newdf['times']=time_bins
newdf.set_index('times',inplace=True)
newdf.transpose().to_csv("codes-by-time-bucket.csv")

## Generate detailed edge list for people network

In [82]:
#Clean up a bit
df['Sender']=df['user_username']
edges_verbose=df[df['Receiver']!=''].copy()
edges_verbose=edges_verbose[edges_verbose['Receiver'].str.contains('Error')==False]
themes = ['Self-organization','Building community','Broadening the counterpublic','Creating change in math','SJEDI']
for column in themes:
    edges_verbose[column]=edges_verbose[column].apply(bool)
cols=['Sender','Receiver','text','created_at']+themes+codes.tolist()+time_bins
#edges_verbose[cols].head(20)

edges_verbose[cols].to_csv("people-edge-list.csv")


